In [127]:
%matplotlib inline

In [128]:
import numpy as np
import tvm
from tvm import te

# The sizes of inputs and filters
batch =1 
img_size = 227
img_ch = 3
filter_size = 3
filter_num = 64

# Algorithm
In = te.placeholder((batch, img_size, img_size, img_ch), name="In")
W1 = te.placeholder((filter_num,filter_size, filter_size, img_ch), name="W1")
W2 = te.placeholder((filter_num, filter_size, filter_size, filter_num), name="W2")

print(In.shape)
print(W1.shape)
print(W2.shape)

[1, 227, 227, 3]
[64, 3, 3, 3]
[64, 3, 3, 64]


In [129]:
def Conv(Input, Kernel, stride, pad, name):
    batch_size, in_size, _, in_channel = Input.shape
    out_channel, kernel_size, _, _ = Kernel.shape

    out_size = (in_size - kernel_size + 2 * pad) // stride + 1
    # Pad input

    # if pad > 0:
    #     Input = te.compute(
    #         (in_size + 2 * pad, in_size + 2 * pad, in_channel, batch_size),
    #         lambda yy, xx, cc, nn: tvm.tir.if_then_else(
    #             tvm.tir.all(yy >= pad, yy - pad < in_size, xx >= pad, xx - pad < in_size),
    #             Input[yy - pad, xx - pad, cc, nn],
    #             tvm.tir.const(0.0, "float32"),),
    #         name=name + "_pad",)
    # Create reduction variables
    rc = te.reduce_axis((0, in_channel), name=name+"_rc")
    ry = te.reduce_axis((0, kernel_size), name=name+"_ry")
    rx = te.reduce_axis((0, kernel_size), name=name+"_rx")
    # Compute the convolution
    Conv = te.compute(
        (batch_size, out_size, out_size, out_channel),
        lambda nn, yy, xx, ff: te.sum(
            Input[nn, yy * stride + ry, xx * stride + rx, rc] * Kernel[ff, ry, rx, rc], axis=[ry, rx, rc]
        ),
        name=name+'_conv',
    )
    return Conv

In [122]:
conv1 = Conv(In, W1, stride=2, pad=0, name='l1')
conv2 = Conv(conv1, W2, stride=2, pad=0, name='l2')

### Origin Schedule

In [123]:
s = te.create_schedule(conv2.op)
print(tvm.lower(s, [In, W1, W2, conv2], simple_mode=True))

@main = primfn(In_1: handle, W1_1: handle, W2_1: handle, l2_conv_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {In: Buffer(In_2: Pointer(float32), float32, [1, 227, 227, 3], []),
             W1: Buffer(W1_2: Pointer(float32), float32, [64, 3, 3, 3], []),
             W2: Buffer(W2_2: Pointer(float32), float32, [64, 3, 3, 64], []),
             l2_conv: Buffer(l2_conv_2: Pointer(float32), float32, [1, 56, 56, 64], [])}
  buffer_map = {In_1: In, W1_1: W1, W2_1: W2, l2_conv_1: l2_conv} {
  allocate(l1_conv: Pointer(global float32), float32, [817216]), storage_scope = global {
    for (yy: int32, 0, 113) {
      for (xx: int32, 0, 113) {
        for (ff: int32, 0, 64) {
          l1_conv_1: Buffer(l1_conv, float32, [817216], [])[(((yy*7232) + (xx*64)) + ff)] = 0f32
          for (l1_ry: int32, 0, 3) {
            for (l1_rx: int32, 0, 3) {
              for (l1_rc: int32, 0, 3) {
                let cse_var_2: int32 =

In [124]:
# s = te.create_schedule(conv2.op)
# xo, yo, xi, yi = s[conv2].tile(conv2.op.axis[0], conv2.op.axis[1], x_factor=28, y_factor=28)
# print(tvm.lower(s, [In, W1, W2, conv2], simple_mode=True))

In [125]:
s = te.create_schedule(conv2.op)
# s[conv1].compute_at(s[conv2], conv2.op.axis[2])
# s[conv1].compute_inline()  # compute Apad inline

SI = s.cache_read(In, "shared", [conv1])
SW1 = s.cache_read(W1, "shared", [conv1])
LI = s.cache_read(SI, "local", [conv1])
LW1 = s.cache_read(SW1, "local", [conv1])
LC1 = s.cache_write(conv1, "shared")

SA = s.cache_read(conv1, "shared", [conv2])
SW2 = s.cache_read(W2, "shared", [conv2])
LA = s.cache_read(SA, "local", [conv2])
LW2 = s.cache_read(SW2, "local", [conv2])
LC2 = s.cache_write(conv2, "local")


print(tvm.lower(s, [In, W1, W2, conv2], simple_mode=True))

@main = primfn(In_1: handle, W1_1: handle, W2_1: handle, l2_conv_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {In: Buffer(In_2: Pointer(float32), float32, [1, 227, 227, 3], []),
             W1: Buffer(W1_2: Pointer(float32), float32, [64, 3, 3, 3], []),
             W2: Buffer(W2_2: Pointer(float32), float32, [64, 3, 3, 64], []),
             l2_conv: Buffer(l2_conv_2: Pointer(float32), float32, [1, 56, 56, 64], [])}
  buffer_map = {In_1: In, W1_1: W1, W2_1: W2, l2_conv_1: l2_conv} {
  allocate(In.shared: Pointer(shared float32), float32, [817216]), storage_scope = shared;
  allocate(In.shared.local: Pointer(local float32), float32, [154587]), storage_scope = local;
  allocate(W1.shared: Pointer(shared float32), float32, [1728]), storage_scope = shared;
  allocate(W1.shared.local: Pointer(local float32), float32, [1728]), storage_scope = local;
  allocate(l1_conv: Pointer(global float32), float32, [817216]), stor

In [126]:
# tile consts
tile = 8
num_thread = 8
block_factor = tile * num_thread
step = 8
vthread = 2

s = te.create_schedule(conv2.op)
# SI = s.cache_read(In, "shared", [conv1])
# SW1 = s.cache_read(W1, "shared", [conv1])
# LI = s.cache_read(SI, "local", [conv1])
# LW1 = s.cache_read(SW1, "local", [conv1])
# LC1 = s.cache_write(conv1, "shared")

# SA = s.cache_read(conv1, "shared", [conv2])
# SW2 = s.cache_read(W2, "shared", [conv2])
# LA = s.cache_read(SA, "local", [conv2])
# LW2 = s.cache_read(SW2, "local", [conv2])
# LC2 = s.cache_write(conv2, "local")

s[conv1].compute_at(s[conv2], conv2.op.axis[2])
s[conv1].compute_inline()  # compute Apad inline

# Get the GPU thread indices
block_x = te.thread_axis("blockIdx.x")
block_y = te.thread_axis("blockIdx.y")
block_z = te.thread_axis("blockIdx.z")
thread_x = te.thread_axis((0, num_thread), "threadIdx.x")
thread_y = te.thread_axis((0, num_thread), "threadIdx.y")
thread_xz = te.thread_axis((0, vthread), "vthread", name="vx")
thread_yz = te.thread_axis((0, vthread), "vthread", name="vy")

# Split the workloads
hi, wi, fi, ni = s[conv2].op.axis
bz = s[conv2].fuse(hi, wi)
by, fi = s[conv2].split(fi, factor=block_factor)
bx, ni = s[conv2].split(ni, factor=block_factor)

# Bind the iteration variables to GPU thread indices
s[conv2].bind(bz, block_z)
s[conv2].bind(by, block_y)
s[conv2].bind(bx, block_x)



print(tvm.lower(s, [In, W1, W2, conv2], simple_mode=True))

@main = primfn(In_1: handle, W1_1: handle, W2_1: handle, l2_conv_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {In: Buffer(In_2: Pointer(float32), float32, [1, 227, 227, 3], []),
             W1: Buffer(W1_2: Pointer(float32), float32, [64, 3, 3, 3], []),
             W2: Buffer(W2_2: Pointer(float32), float32, [64, 3, 3, 64], []),
             l2_conv: Buffer(l2_conv_2: Pointer(float32), float32, [1, 56, 56, 64], [])}
  buffer_map = {In_1: In, W1_1: W1, W2_1: W2, l2_conv_1: l2_conv} {
  attr [IterVar(blockIdx.z: int32, (nullptr), "ThreadIndex", "blockIdx.z")] "thread_extent" = 56;
  attr [IterVar(blockIdx.y: int32, (nullptr), "ThreadIndex", "blockIdx.y")] "thread_extent" = 1;
  for (xx.inner: int32, 0, 56) {
    attr [IterVar(blockIdx.x: int32, (nullptr), "ThreadIndex", "blockIdx.x")] "thread_extent" = 1;
    for (ff.inner: int32, 0, 64) {
      l2_conv_3: Buffer(l2_conv_2, float32, [200704], [])[(((blockIdx.z*358

In [22]:
# tile consts
tile = 8
num_thread = 8
block_factor = tile * num_thread
step = 8
vthread = 2

s = te.create_schedule(conv2.op)
SI = s.cache_read(In, "shared", [conv1])
SW1 = s.cache_read(W1, "shared", [conv1])
LI = s.cache_read(SI, "local", [conv1])
LW1 = s.cache_read(SW1, "local", [conv1])
LC1 = s.cache_write(conv1, "shared")

SA = s.cache_read(conv1, "shared", [conv2])
SW2 = s.cache_read(W2, "shared", [conv2])
LA = s.cache_read(SA, "local", [conv2])
LW2 = s.cache_read(SW2, "local", [conv2])
LC2 = s.cache_write(conv2, "local")

s[conv1].compute_at(s[conv2], conv2.op.axis[2])
s[conv1].compute_inline()  # compute Apad inline

# Get the GPU thread indices
block_x = te.thread_axis("blockIdx.x")
block_y = te.thread_axis("blockIdx.y")
block_z = te.thread_axis("blockIdx.z")
thread_x = te.thread_axis((0, num_thread), "threadIdx.x")
thread_y = te.thread_axis((0, num_thread), "threadIdx.y")
thread_xz = te.thread_axis((0, vthread), "vthread", name="vx")
thread_yz = te.thread_axis((0, vthread), "vthread", name="vy")

# Split the workloads
hi, wi, fi, ni = s[conv2].op.axis
bz = s[conv2].fuse(hi, wi)
by, fi = s[conv2].split(fi, factor=block_factor)
bx, ni = s[conv2].split(ni, factor=block_factor)

# Bind the iteration variables to GPU thread indices
s[conv2].bind(bz, block_z)
s[conv2].bind(by, block_y)
s[conv2].bind(bx, block_x)



tyz, fi = s[conv2].split(fi, nparts=vthread)  # virtual thread split
txz, ni = s[conv2].split(ni, nparts=vthread)  # virtual thread split
ty, fi = s[conv2].split(fi, nparts=num_thread)
tx, ni = s[conv2].split(ni, nparts=num_thread)
s[conv2].reorder(bz, by, bx, tyz, txz, ty, tx, fi, ni)

s[conv2].bind(tyz, thread_yz)
s[conv2].bind(txz, thread_xz)
s[conv2].bind(ty, thread_y)
s[conv2].bind(tx, thread_x)

print(tvm.lower(s, [In, W1, W2, conv2], simple_mode=True))

@main = primfn(In_1: handle, W1_1: handle, W2_1: handle, l2_conv_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {In: Buffer(In_2: Pointer(float32), float32, [227, 227, 3, 1], []),
             W1: Buffer(W1_2: Pointer(float32), float32, [3, 3, 3, 64], []),
             W2: Buffer(W2_2: Pointer(float32), float32, [3, 3, 64, 64], []),
             l2_conv: Buffer(l2_conv_2: Pointer(float32), float32, [56, 56, 64, 1], [])}
  buffer_map = {In_1: In, W1_1: W1, W2_1: W2, l2_conv_1: l2_conv} {
  allocate(In.shared: Pointer(shared float32), float32, [2304]), storage_scope = shared;
  allocate(In.shared.local: Pointer(local float32), float32, [588]), storage_scope = local;
  allocate(W1.shared.local: Pointer(local float32), float32, [1728]), storage_scope = local;
  allocate(l1_conv.d.shared.local: Pointer(local float32), float32, [2304]), storage_scope = local;
  allocate(W2.shared.local: Pointer(local float32), float32, [2

In [26]:
# tile consts
tile = 8
num_thread = 8
block_factor = tile * num_thread
step = 8
vthread = 2

s = te.create_schedule(conv2.op)
SI = s.cache_read(In, "shared", [conv1])
SW1 = s.cache_read(W1, "shared", [conv1])
LI = s.cache_read(SI, "local", [conv1])
LW1 = s.cache_read(SW1, "local", [conv1])
LC1 = s.cache_write(conv1, "shared")

SA = s.cache_read(conv1, "shared", [conv2])
SW2 = s.cache_read(W2, "shared", [conv2])
LA = s.cache_read(SA, "local", [conv2])
LW2 = s.cache_read(SW2, "local", [conv2])
LC2 = s.cache_write(conv2, "local")

s[conv1].compute_at(s[conv2], conv2.op.axis[2])
s[conv1].compute_inline()  # compute Apad inline

# Get the GPU thread indices
block_x = te.thread_axis("blockIdx.x")
block_y = te.thread_axis("blockIdx.y")
block_z = te.thread_axis("blockIdx.z")
thread_x = te.thread_axis((0, num_thread), "threadIdx.x")
thread_y = te.thread_axis((0, num_thread), "threadIdx.y")
thread_xz = te.thread_axis((0, vthread), "vthread", name="vx")
thread_yz = te.thread_axis((0, vthread), "vthread", name="vy")

# Split the workloads
hi, wi, fi, ni = s[conv1].op.axis
bz = s[conv1].fuse(hi, wi)
by, fi = s[conv1].split(fi, factor=block_factor)
bx, ni = s[conv1].split(ni, factor=block_factor)

# Bind the iteration variables to GPU thread indices
s[conv1].bind(bz, block_z)
s[conv1].bind(by, block_y)
s[conv1].bind(bx, block_x)

# Split the workloads
hi, wi, fi, ni = s[conv2].op.axis
bz = s[conv2].fuse(hi, wi)
by, fi = s[conv2].split(fi, factor=block_factor)
bx, ni = s[conv2].split(ni, factor=block_factor)

# Bind the iteration variables to GPU thread indices
s[conv2].bind(bz, block_z)
s[conv2].bind(by, block_y)
s[conv2].bind(bx, block_x)


tyz, fi = s[conv2].split(fi, nparts=vthread)  # virtual thread split
txz, ni = s[conv2].split(ni, nparts=vthread)  # virtual thread split
ty, fi = s[conv2].split(fi, nparts=num_thread)
tx, ni = s[conv2].split(ni, nparts=num_thread)
s[conv2].reorder(bz, by, bx, tyz, txz, ty, tx, fi, ni)

s[conv2].bind(tyz, thread_yz)
s[conv2].bind(txz, thread_xz)
s[conv2].bind(ty, thread_y)
s[conv2].bind(tx, thread_x)

# Schedule BL local write
s[LC2].compute_at(s[conv2], tx)
yi, xi, fi, ni = s[LC2].op.axis
ry, rx, rc = s[LC2].op.reduce_axis
rco, rci = s[LC2].split(rc, factor=step)
s[LC2].reorder(rco, ry, rx, rci, fi, ni)

# # Attach computation to iteration variables
# s[AA].compute_at(s[BL], rx)
# s[WW].compute_at(s[BL], rx)
# s[AL].compute_at(s[BL], rci)
# s[WL].compute_at(s[BL], rci)

# # Schedule for A's shared memory load
yi, xi, ci, ni = s[conv1].op.axis
ty, ci = s[conv1].split(ci, nparts=num_thread)
tx, ni = s[conv1].split(ni, nparts=num_thread)
_, ni = s[conv1].split(ni, factor=4)
s[conv1].reorder(ty, tx, yi, xi, ci, ni)
s[conv1].bind(ty, thread_y)
s[conv1].bind(tx, thread_x)
s[conv1].vectorize(ni)  # vectorize memory load

# # Schedule for W's shared memory load
# yi, xi, ci, fi = s[WW].op.axis
# ty, ci = s[WW].split(ci, nparts=num_thread)
# tx, fi = s[WW].split(fi, nparts=num_thread)
# _, fi = s[WW].split(fi, factor=4)
# s[WW].reorder(ty, tx, yi, xi, ci, fi)
# s[WW].bind(ty, thread_y)
# s[WW].bind(tx, thread_x)
# s[WW].vectorize(fi)  # vectorize memory load

print(tvm.lower(s, [In, W1, W2, conv2], simple_mode=True))

TVMError: Traceback (most recent call last):
  6: TVMFuncCall
  5: _ZN3tvm7runtime13PackedFuncObj
  4: tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::te::Schedule, tvm::runtime::Array<tvm::runtime::ObjectRef, void> const&, tvm::runtime::String const&, tvm::runtime::Map<tvm::te::Tensor, tvm::tir::Buffer, void, void> const&, bool)>::AssignTypedLambda<tvm::{lambda(tvm::te::Schedule, tvm::runtime::Array<tvm::runtime::ObjectRef, void> const&, tvm::runtime::String const&, tvm::runtime::Map<tvm::te::Tensor, tvm::tir::Buffer, void, void> const&, bool)#5}>(tvm::{lambda(tvm::te::Schedule, tvm::runtime::Array<tvm::runtime::ObjectRef, void> const&, tvm::runtime::String const&, tvm::runtime::Map<tvm::te::Tensor, tvm::tir::Buffer, void, void> const&, bool)#5}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}::operator()(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*) const
  3: tvm::Low